In [ ]:
#******************************************************************************************************************************************************#
"""
    @author Shaela Khan, Created 1st April,2022,  Updated : 24th April, Sunday, Last Updated 19th August, Friday 2022.

#  song_recognizer.ipynb  -> In Modelling Phase
#  Bird Recognition from birdsongs using Deep learning -> We are working on building a model using Deep learning techniques to identify birds from their songs.
#  DataSource : - https://www.kaggle.com/datasets/rtatman/british-birdsong-dataset  (Aggregated from the original Xeno-Canto Dataset.)
#  Provided dataset has a directory with - ./small Xeno-Canto/songs --> Original audio dataset.
#                                        -  ./small Xeno-Canto/birdsong_metadata.csv  --> Original Dataset with labels.
                                         -   ./img_data --> audio files converted into image files --> contains spectrographs of audio data.
#                                        - traincsv.csv file with labels
#                                        - testcsv.csv - for testing model performance
#
#  We then create a CNN model with possible usage of pre-trained models, that can identify the difference classes defined - this is a supervised learning
#  problem.
#  Input: birdsong_metadata.csv + songs
#  Output : The file should have
#          : Classify image from testing provided.
"""
#*******************************************************************************************************************************************************#

In [ ]:

# COMMON LIBRARIES
import os,sys,random,math
from random import seed, random, randint
from tqdm import tqdm
import pathlib
from pathlib import Path
# import import_ipynb,import nbimporter , import helper-functions


# FILE PROCESSING
import csv
import pandas as pd
import shutil
from shutil import move
from itertools import groupby

# Plotting AND Visuals.
import matplotlib.pyplot as plt
import numpy as np, shutil, itertools, pickle,seaborn as sn, math, time,scipy

# Stats
from scipy.spatial import distance
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
from mpl_toolkits.mplot3d import Axes3D
from scipy import spatial


# Image processing libraries
#!pip install Pillow
from PIL import Image


# EXTRA -
%matplotlib inline

# ML, statistics
import warnings
warnings.filterwarnings('ignore')
import scipy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.decomposition import PCA
#from sklearn.preprocessing import OneHotEncoder


# TENSORFLOW LIBRARIES. BACK-END SUPPORT.
# Creating Model and model Evaluation
import tensorflow as tf
import tensorboard,tensorflow_estimator
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.layers import GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint

import tensorflow.keras.utils
import tensorflow.keras.activations
import tensorflow.keras.applications
import tensorflow.keras.callbacks
import tensorflow.keras.initializers
from tensorflow.keras import losses,metrics


# Audio file processing
import librosa
import librosa.display
import soundfile as sf


# EXTRA EXTRA - because they don't work upstairs?
import pydub
from pydub import AudioSegment
import IPython
from IPython.display import Audio
warnings.filterwarnings('ignore')


# Start Code Here.
print("Hello World ! ")
print("**********  Sweet Caroline ba ba ba, Shit hehehe!!**********")
print("\n Tensorflow version: ", tf.__version__)

# Prints current Python version
print("Python  ", sys.version)
#print("\n Pillow " )

In [ ]:
# Data preprocessing.****************************************************
# Read metadata file
metadata = pd.read_csv("./small Xeno-Canto/birdsong_metadata.csv") # metadata pandas object Holds all of the data from the csv file.
header = list(metadata.head())


# Get bird names
bird_name = metadata['english_cname'].values
u, f = np.unique(bird_name, return_counts=True)
print("Unique Bird Names  \n",u) # Dont touch this part - works
print("data structure.")
print(metadata.head())
print(metadata.info(verbose=True))
#print("\n", bird_name)

# helper function, to sort in alphabetical order.
#guest_birds = bird_name
#guest_birds.sort()
#print("Hoopla over nothing. ",guest_birds)


In [ ]:
from tokenize import String
# ADDING labels now , from the csv files - pandas provides the df object.
from keras_preprocessing.image import ImageDataGenerator
tdf = pd.read_csv('./small Xeno-Canto/birdsong_metadata.csv')
#print(tdf.head())
#print(tdf.info(verbose=True))


# Cleanup tdf  tdf  ----> According to proper processing procedures.
tdf = tdf.drop(columns='who_provided_recording')
tdf = tdf.drop(columns='country')
tdf = tdf.drop(columns='latitude')
tdf = tdf.drop(columns='longitute')
tdf = tdf.drop(columns='license')
#print("After dropping the redundant columns.")  #print(tdf.head())
# Find a better way to drop multiple columns drop dataframe columns.


# For brevity and clarity in processing.
tdf['file_id'] = tdf['file_id'].apply(str)


# train_df = The one here is a manual hack. Cleaned up the metadata.csv manually.
train_df = pd.read_csv('./small Xeno-Canto/birdsong_metadata_modified.csv')
train_df.fillna(0, inplace=True)

def extension_train_data(x):
    return "xc"+str(x)+".png"

# CHANGING the File extensions from .wav to png --> If you remember the originals were audio files.
train_df['file_id'] = train_df['file_id'].apply(extension_train_data)
print("Checking......................")
print(train_df.head())

In [ ]:
#Pre-processing test-dataset and creating a pandas dataframe for ease of IMPLEMENTATION.
print("\n Pre-processing test-dataset and creating a pandas dataframe for ease of IMPLEMENTATION. ")
sdir = "./img_data-test"
filepaths=[]
labels=[]
flist=os.listdir(sdir)

for f in flist:
    fpath=os.path.join(sdir,f)
    filepaths.append(fpath)
    index1=f.rfind('-')+1
    index2=f.rfind('.')
    klass=f[index1:index2]
    labels.append(klass)
Fseries=pd.Series(filepaths, name='filepaths')
Lseries=pd.Series(labels, name='labels')
test_df=pd.concat([Fseries, Lseries], axis=1)
#print(test_df.head())
#print("\n ", labels)
print("\nChecking for TEST dataset  ......................")


In [ ]:
# Creating data generators.
# rescale all pixel values from 0-255, so after this step all our pixel values are in range (0,1)
print("Create DATA generators for the model.\n")

datagen=ImageDataGenerator(rescale=1./255,validation_split=0.3) # Training and Validation split 70/30
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator =datagen.flow_from_dataframe(dataframe=train_df,directory='./img_data/',
                                             x_col="file_id",
                                             y_col="english_cname",
                                             class_mode="categorical",
                                             target_size=(64,64),
                                             validation_split=0.2,
                                             subset="training",
                                             seed=1337,batch_size=32,shuffle=False)

print('Train generator created')
val_generator =datagen.flow_from_dataframe(dataframe=train_df,directory='./img_data/',
                                           x_col="file_id",
                                           y_col="english_cname",
                                           class_mode="categorical",
                                           target_size=(64,64),subset="validation",
                                           seed=42,batch_size=32,shuffle=False)


print('Validation generator created')
# test_set =test_datagen.flow_from_directory(test_df,target_size=(64, 64),
                                          # batch_size=32, class_mode='categorical', shuffle = False)

test_generator=test_datagen.flow_from_dataframe(test_df,
                                                x_col='filepaths',
                                                y_col='labels',
                                                target_size=(64,64),
                                                class_mode='categorical',
                                                color_mode='rgb', shuffle=False, batch_size=2)

# test_gen=tvgen.flow_from_dataframe(test_df, x_col='filepaths', y_col='labels',target_size=img_size,
                                  # class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=2)




print('Test generator created')
print("\n Sanity check Line.----------")
#print("Printing labels of the validation dataset: \n",val_generator.labels)
#print("Printing label- names  of the validation dataset: \n",val_generator.class_indices.keys())
#print("\n These names are needed for report and Presentation.")

In [ ]:
# Helper function- to be moved to helper-functions.ipynb
from datetime import datetime
def timer():
    # datetime object containing current date and time
    now = datetime.now()
    print("now =", now)
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print("date and time =", dt_string)

print("\n DONE.")

In [ ]:

# Model IMPLEMENTATIOM
print("Creating simple CNN model \n")
model_simple = tf.keras.models.Sequential()
#model_tn = Sequential()

input_shape=(64, 64, 3)#1st hidden layer
model_simple.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
model_simple.add(AveragePooling2D((2, 2), strides=(2,2)))
model_simple.add(Activation('relu'))#2nd hidden layer
model_simple.add(Conv2D(64, (3, 3), padding="same"))
model_simple.add(AveragePooling2D((2, 2), strides=(2,2)))
model_simple.add(Activation('relu'))#3rd hidden layer
model_simple.add(Conv2D(64, (3, 3), padding="same"))
model_simple.add(AveragePooling2D((2, 2), strides=(2,2)))
model_simple.add(Activation('relu'))#Flatten
model_simple.add(Flatten())
model_simple.add(Dropout(rate=0.5))#Add fully connected layer.
model_simple.add(Dense(64))
model_simple.add(Activation('relu'))
model_simple.add(Dropout(rate=0.5))#Output layer
model_simple.add(Dense(88))   # NUMBER OF CLASSES TO PREDICT.
model_simple.add(Activation('softmax'))
model_simple.summary()




In [ ]:

# MODEL Specifications SETUP.
""" Model specifications notes = these specs epochs= 200, batch_size=8 learning rate=0.01, momentum= 0.9 SGD--> accuracy was shit."""
EPOCHS = 200
BATCH_SIZE = 8
l1_learning_rate = 0.01
decay_rate = l1_learning_rate / EPOCHS
momentum = 0.9
sgd = SGD(lr=l1_learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
adam = Adam(learning_rate=l1_learning_rate)
model_simple.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])
print("\n Done. ")

In [ ]:

# Fit the training-set to the model and Train.
train_set = train_generator
validation_set = val_generator
test_set  = test_generator

history = model_simple.fit( train_set,validation_data=validation_set,epochs=EPOCHS,batch_size=BATCH_SIZE)
# model._reset_compile_cache()  #print(test_set)
#model_simple.fit_generator(train_set, steps_per_epoch=100,epochs=50,validation_data=validation_set,validation_steps=50)
print("\n Training Done. ")
print("\n Metrics: ",history.history)


In [ ]:
#Model Evaluation
print("Evaluate on test data")
results = model_simple.evaluate(train_set,test_set, batch_size=128)#OUTPUT [1.704445120342617, 0.33798882681564246]
print("test loss, test acc:", results)

test_set.reset()
pred = model_simple.predict(test_set, steps=50, verbose=1)

In [ ]:

print("So, training Data is now processed. Omg! So much work. Anyway -- now we need to Create the test data folder. Pre-process that and then feed it to cnn ANd then pre-trained model . Evaluate etc...")
print("\n Restarted : -----------------------------------------------------------------")
print(" Creating test dataset complete! Pre-processing done, TIME FOR MODELLING ")
timer()
print("\n Initial modelling and training complete.")

print("***********************************  End of Processing. !!************************************************")

In [ ]:
# Using PRE-Trained network Models.
#import keras_efficientnets from keras_efficientnets import EfficientNetB5
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input


print("Using ResNet50 network Models---------------------------------------------------")
img_size=(224,224)
batch_size=16
# build a model
img_shape=(img_size[0], img_size[1], 3)
"""model_name="ResNet50"
base_model=tf.keras.applications.resnet50.ResNet50(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
#babyURAModel = tf.keras.applications.EfficientNetB5()


base_model=tf.keras.applications.ResNet50(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')
# Note you are always told NOT to make the base model trainable initially, that is WRONG you get better results leaving it trainable
base_model.trainable=True
x=base_model.output
x=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(1024, kernel_regularizer = tf.keras.regularizers.l2(l = 0.016),activity_regularizer=tf.keras.regularizers.l1(0.006),
                bias_regularizer=tf.keras.regularizers.l1(0.006) ,activation='relu')(x)
x=Dropout(rate=.3, seed=123)(x) # Dropout(rate=0.3, seed=123)(x)
x = Dense(128, kernel_regularizer = tf.keras.regularizers.l2(l = 0.016),activity_regularizer=tf.keras.regularizers.l1(0.006),
                bias_regularizer=tf.keras.regularizers.l1(0.006) ,activation='relu')(x)
x=Dropout(rate=.45, seed=123)(x)
output=Dense(class_count, activation='softmax')(x)  # Output Layer= class_count specifies how many classes it's supposed to classify.

model=Model(inputs=base_model.input, outputs=output)
# OPTIMIZERS
lr=.001 # start with this learning rate
sgd =tf.keras.optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
#model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])


model.compile(tf.keras.optimizers.Adamax(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary() """

input_shape = (216,216, 3)
resnet_layers = tf.keras.applications.ResNet50(weights=None, include_top=False, input_shape=input_shape)

for layer in resnet_layers.layers:
    layer.trainable = True

dropout_dense_layer = 0.3
model = Sequential()
model.add(resnet_layers)

model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Dense(256, use_bias=False))
model.add(tf.keras.layers.BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dropout_dense_layer))
model.add(Dense(88, activation="softmax"))

model.summary()

print("-- Model Architecture--\n")

In [ ]:
# Train Model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
#NUM_EPOCHS = 10
#EARLY_STOP_PATIENCE = 3
# Early stopping & checkpointing the best model in ../working dir & restoring that as our model for prediction
#cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
#cb_checkpointer = ModelCheckpoint(filepath = './models/best.hdf5', monitor='val_loss',
#                                  save_best_only = True, mode = 'auto')
epoch10 = 10
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.7),
             #EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint(filepath='./models/best.hdf5', monitor='val_loss', save_best_only=True, mode='auto')]
model.compile(loss="categorical_crossentropy",metrics=['accuracy'], optimizer='adam')

history = model.fit(train_generator,
                    epochs = 100,
                    validation_data=val_generator,
                    verbose=True,batch_size=32,
                    #class_weight=class_weights_dict,
                    callbacks=[callbacks])